# Keras colorizer of CelebA using Generative Adversarial Networks.
The dataset can be downloaded from: https://www.dropbox.com/sh/8oqt9vytwxb3s4r/AADIKlz8PR9zr6Y20qbkunrba/Img/img_align_celeba.zip?dl=0
## Instrustion on running the script:
1. Download the dataset from the provided link
2. Save the folder 'img_align_celeba' to 'datasets/'

In [ ]:
! python

In [2]:
tf.__version__

'1.14.0'

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! mkdir datasets
! mkdir originals

In [ ]:
! unzip -q "/content/drive/My Drive/img_align_celeba.zip" -d datasets

In [ ]:
! unzip -q "/content/drive/My Drive/coco_val2017.zip" -d datasets

In [ ]:
! mv datasets/val2017/* originals

In [ ]:
! find /content/datasets/img_align_celeba -type f -exec sh -c 'mv "$@" "$0"' originals/ {} +

In [23]:
! pip install segmentation-models

In [1]:
import scipy
import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add, MaxPooling2D
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
import keras
#from tensorflow.keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
from glob import glob
import keras.backend as K
import scipy.misc
import PIL
import os
import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
from keras.optimizers import Adam
import segmentation_models as sm

D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

Segmentation Models: using `keras` framework.


In [2]:
import keras
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 3} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [4]:
gen_192 = keras.models.load_model('generator.h5')
gen_192

D:\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [8]:
gen_192 = gen_192.layers[0]

In [9]:
gen_192.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 192, 192, 1)  0                                            
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 192, 192, 64) 640         input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 192, 192, 64) 36928       conv2d_41[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 96, 96, 64)   0           conv2d_42[0][0]                  
____________________________________________________________________________________________

In [30]:
gen = keras.models.Sequential()
gen.add(Conv2D(64, kernel_size=(3, 3), input_shape=(384, 384, 1), activation='relu', padding='same'))
gen.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
gen.add(MaxPooling2D(pool_size=(2, 2), name="kek"))


In [31]:
for i, layer in enumerate(gen_192.layers[2:]):
    gen.add(layer)

ValueError: A merge layer should be called on a list of inputs.

In [27]:
gen.add?

In [15]:
gen.build()

In [16]:
gen.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 384, 384, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 384, 384, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 192, 192, 64)      0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 192, 192, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 96, 96, 64)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 96, 96, 128)       73856     
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 96, 96, 128)      

In [15]:
class DataLoader():
  def __init__(self, img_res=(256, 256)):
    self.img_res = img_res

  def load_data(self, batch_size=1, is_testing=False):
    path = glob('./originals/*')
    batch_images = np.random.choice(path, size=batch_size)

    imgs_hr = []
    imgs_lr = []
    for img_path in batch_images:
      img_hr, img_lr = self._load(img_path, self.img_res)

      imgs_hr.append(img_hr)
      imgs_lr.append(img_lr)
      
    # нормализация данных
    imgs_hr = np.array(imgs_hr) / 127.5 - 1
    imgs_lr = np.array(imgs_lr) / 127.5 - 1

    return imgs_hr, imgs_lr

  # returns pair (original photo, grayscale photo)
  def _load(self, path, size):
    img = PIL.Image.open(path).resize(size).convert('RGB')
    orig = np.array(img).astype(np.float)
    gray = np.expand_dims(np.array(img.convert('L')).astype(np.float), axis=2)
    # np.expand_dims(np.array(PIL.Image.open(path).resize(size).convert('L')).astype(np.float), axis=2)
    return orig, gray

In [16]:
hr_channels = 3
lr_channels = 1
width = 192
lr_shape = (width, width, lr_channels)
hr_shape = (width, width, hr_channels)

n_residual_blocks = 16
optimizer = Adam(0.0002, 0.5)

# configure data loader
data_loader = DataLoader(img_res=(width, width))

In [24]:
def sample_images(epoch):
  os.makedirs('samples', exist_ok=True)
  r,c = 2,3
  imgs_hr, imgs_lr = data_loader.load_data(batch_size=2, is_testing=True)
  fake_hr = generator.predict(imgs_lr)
  imgs_lr = 0.5 * imgs_lr + 0.5
  fake_hr = 0.5 * fake_hr + 0.5
  imgs_hr = 0.5 * imgs_hr + 0.5
  titles = ["B&W", "Generated", "Original"]
  fig, axs = plt.subplots(r, c)
  cnt = 0
  for row in range(r):
    for col, image in enumerate([imgs_lr, fake_hr, imgs_hr]):
        if col == 0:
            axs[row, col].imshow(np.array(PIL.Image.fromarray((np.squeeze(image[row])*255).astype(np.uint8)).convert('RGB')))
        else:
            axs[row, col].imshow(image[row])
        axs[row, col].set_title(titles[col])
        axs[row, col].axis('off')
    cnt += 1
  def zerofy(s: str):
    while len(s) < 6:
      s = '0' + s
    return s
  fig.savefig('samples/'+zerofy(str(epoch)) +'.png')
  plt.close()

We use a pre-trained VGG19 model to extract image features from the high resolution and the generated high resolution images and minimize the mse between them

In [18]:
def build_feature_loss(input_size):
  vgg = VGG19(weights='imagenet')
  vgg.outputs = [vgg.layers[9].output]
  img = Input(shape=input_size)
  img_features = vgg(img)
  model = Model(img, img_features)
  model.trainable = False 
  model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
  return model
feature_loss = build_feature_loss(hr_shape)

build and compile the discriminator

In [19]:
def build_discriminator(input_size):
  # Number of filters in the first layer of G and D
  gf = 64
  df = 64
  def d_block(layer_input, filters, strides=1, bn=True):
    """Discriminator layer"""
    d = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    if bn:
        d = BatchNormalization(momentum=0.8)(d)
    return d

  # Input img
  d0 = Input(shape=input_size)

  d1 = d_block(d0, df, bn=False)
  d2 = d_block(d1, df, strides=2)
  d3 = d_block(d2, df*2)
  d4 = d_block(d3, df*2, strides=2)
  d5 = d_block(d4, df*4)
  d6 = d_block(d5, df*4, strides=2)
  d7 = d_block(d6, df*8)
  d8 = d_block(d7, df*8, strides=2)

  d9 = Dense(df*16)(d8)
  d10 = LeakyReLU(alpha=0.2)(d9)
  validity = Dense(1, activation='sigmoid')(d10)

  model = Model(d0, validity)
  model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
  return model

discriminator = build_discriminator(input_size=hr_shape)

Build the generator

In [20]:
def unet(pretrained_weights = None, input_size = (256, 256, 3)):
  inputs = Input(input_size)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = Dropout(0.5)(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv9 = Conv2D(3, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv10 = Conv2D(3, 1, activation = 'tanh')(conv9)

  model = Model(input = inputs, output = conv10)
  
  #model.summary()

  if(pretrained_weights):
    model.load_weights(pretrained_weights)
  model.compile(optimizer = Adam(lr = 1e-4), loss = 'mse', metrics = ['accuracy'])
  return model
generator = unet(input_size=lr_shape)
#generator = sm.Unet('resnet34', input_shape=(width, width, 3), encoder_weights='imagenet', classes=3, activation="tanh", encoder_freeze=True,decoder_filters=(1024, 512,256, 128, 64))
#generator.compile(optimizer = Adam(lr = 1e-4), loss = 'mse', metrics = ['accuracy'])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [21]:
generator.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 192, 192, 1)  0                                            
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 192, 192, 64) 640         input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 192, 192, 64) 36928       conv2d_41[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 96, 96, 64)   0           conv2d_42[0][0]                  
____________________________________________________________________________________________

In [22]:
def build_gan(input_size):
  # High res. and low res. images
  img_lr = Input(shape=lr_shape)
  # generate high res. version from low res.
  fake_hr = generator(img_lr)
  # extract image features of the generated img
  fake_features = feature_loss(fake_hr)
  # for the combined model we will only train the generator
  discriminator.trainable = False
  # Discriminator determines validity of generated high res. images
  validity = discriminator(fake_hr)
  combined = Model([img_lr], [validity, fake_features, fake_hr])
  combined.compile(loss=['binary_crossentropy', 'mse', 'mse'], loss_weights=[1e-3, 1, 1e-5], optimizer=optimizer)
  return combined
gan = build_gan(lr_shape)

## Train

In [26]:
#! rm -rf ./samples
epochs=100000
batch_size=2
sample_interval=50
start_time = datetime.datetime.now()
for epoch in range(epochs+1):
  # calculate output shape of D (PatchGAN)
  patch = int(width / 2**4)
  disc_patch = (patch, patch, 1)
  # ----------------------
  #  Train Discriminator
  # ----------------------

  # Sample images and their conditioning counterparts
  imgs_hr, imgs_lr = data_loader.load_data(batch_size)

  # From low res. image generate high res. version
  fake_hr = generator.predict(imgs_lr)

  valid = np.ones((batch_size,) + disc_patch)
  fake = np.zeros((batch_size,) + disc_patch)

  # Train the discriminators (original images = real / generated = Fake)
  d_loss_real = discriminator.train_on_batch(imgs_hr, valid)
  d_loss_fake = discriminator.train_on_batch(fake_hr, fake)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # ------------------
  #  Train Generator
  # ------------------

  # Sample images and their conditioning counterparts
  imgs_hr, imgs_lr = data_loader.load_data(batch_size)

  # The generators want the discriminators to label the generated images as real
  valid = np.ones((batch_size,) + disc_patch)

  # Extract ground truth image features using pre-trained VGG19 model
  image_features = feature_loss.predict(imgs_hr)

  # Train the generators
  g_loss = gan.train_on_batch([imgs_lr], [valid, image_features, imgs_hr])

  elapsed_time = datetime.datetime.now() - start_time
  # Plot the progress
  print ("%d time: %s" % (epoch, elapsed_time))

  # If at save interval => save generated image samples
  if epoch % sample_interval == 0:
      sample_images(epoch)

0 time: 0:00:03.731764
1 time: 0:00:05.588412
2 time: 0:00:06.832044
3 time: 0:00:08.092236
4 time: 0:00:09.307846
5 time: 0:00:10.526629
6 time: 0:00:11.754902
7 time: 0:00:12.974926
8 time: 0:00:14.194379
9 time: 0:00:15.404373
10 time: 0:00:16.622420
11 time: 0:00:17.836895
12 time: 0:00:19.056452
13 time: 0:00:20.294647
14 time: 0:00:21.546583
15 time: 0:00:22.767837
16 time: 0:00:23.981705
17 time: 0:00:25.198440
18 time: 0:00:26.416235
19 time: 0:00:27.639317
20 time: 0:00:28.851557
21 time: 0:00:30.063072
22 time: 0:00:31.279599
23 time: 0:00:32.493379
24 time: 0:00:33.701562
25 time: 0:00:34.933731
26 time: 0:00:36.173565
27 time: 0:00:37.415900
28 time: 0:00:38.631646
29 time: 0:00:39.842065
30 time: 0:00:41.054757
31 time: 0:00:42.273244
32 time: 0:00:43.487029
33 time: 0:00:44.701316
34 time: 0:00:45.917609
35 time: 0:00:47.129891
36 time: 0:00:48.350072
37 time: 0:00:49.572550
38 time: 0:00:50.817527
39 time: 0:00:52.067267
40 time: 0:00:53.285548
41 time: 0:00:54.510099
42

333 time: 0:06:57.189984
334 time: 0:06:58.426924
335 time: 0:06:59.652646
336 time: 0:07:00.898631
337 time: 0:07:02.150815
338 time: 0:07:03.365918
339 time: 0:07:04.582717
340 time: 0:07:05.807391
341 time: 0:07:07.028137
342 time: 0:07:08.253361
343 time: 0:07:09.478322
344 time: 0:07:10.703094
345 time: 0:07:11.941728
346 time: 0:07:13.193446
347 time: 0:07:14.414329
348 time: 0:07:15.642186
349 time: 0:07:16.867949
350 time: 0:07:18.081478
351 time: 0:07:19.938536
352 time: 0:07:21.192207
353 time: 0:07:22.461523
354 time: 0:07:23.686802
355 time: 0:07:24.913983
356 time: 0:07:26.135350
357 time: 0:07:27.362070
358 time: 0:07:28.610333
359 time: 0:07:29.862316
360 time: 0:07:31.087087
361 time: 0:07:32.321385
362 time: 0:07:33.550628
363 time: 0:07:34.767825
364 time: 0:07:35.992245
365 time: 0:07:37.237494
366 time: 0:07:38.497185
367 time: 0:07:39.720853
368 time: 0:07:40.945065
369 time: 0:07:42.165661
370 time: 0:07:43.384952
371 time: 0:07:44.603883
372 time: 0:07:45.830855


661 time: 0:13:47.956596
662 time: 0:13:49.190714
663 time: 0:13:50.413977
664 time: 0:13:51.651256
665 time: 0:13:52.913168
666 time: 0:13:54.159627
667 time: 0:13:55.393992
668 time: 0:13:56.625586
669 time: 0:13:57.861335
670 time: 0:13:59.110098
671 time: 0:14:00.369765
672 time: 0:14:01.607725
673 time: 0:14:02.862115
674 time: 0:14:04.122994
675 time: 0:14:05.392179
676 time: 0:14:06.628153
677 time: 0:14:07.863873
678 time: 0:14:09.093256
679 time: 0:14:10.328135
680 time: 0:14:11.603658
681 time: 0:14:12.846616
682 time: 0:14:14.077073
683 time: 0:14:15.299446
684 time: 0:14:16.533333
685 time: 0:14:17.782562
686 time: 0:14:19.050178
687 time: 0:14:20.283172
688 time: 0:14:21.514169
689 time: 0:14:22.745909
690 time: 0:14:23.980273
691 time: 0:14:25.236124
692 time: 0:14:26.492256
693 time: 0:14:27.727235
694 time: 0:14:28.961735
695 time: 0:14:30.195549
696 time: 0:14:31.444713
697 time: 0:14:32.709966
698 time: 0:14:33.950619
699 time: 0:14:35.185110
700 time: 0:14:36.418376


989 time: 0:20:40.417427
990 time: 0:20:41.657617
991 time: 0:20:42.917878
992 time: 0:20:44.163631
993 time: 0:20:45.390997
994 time: 0:20:46.634685
995 time: 0:20:47.868261
996 time: 0:20:49.128776
997 time: 0:20:50.391951
998 time: 0:20:51.621189
999 time: 0:20:52.861717
1000 time: 0:20:54.092438
1001 time: 0:20:55.986592
1002 time: 0:20:57.229577
1003 time: 0:20:58.459845
1004 time: 0:20:59.696403
1005 time: 0:21:00.924348
1006 time: 0:21:02.184537
1007 time: 0:21:03.435981
1008 time: 0:21:04.663743
1009 time: 0:21:05.897383
1010 time: 0:21:07.129568
1011 time: 0:21:08.389711
1012 time: 0:21:09.647761
1013 time: 0:21:10.891828
1014 time: 0:21:12.151443
1015 time: 0:21:13.405229
1016 time: 0:21:14.664635
1017 time: 0:21:15.894948
1018 time: 0:21:17.128564
1019 time: 0:21:18.369246
1020 time: 0:21:19.612018
1021 time: 0:21:20.885660
1022 time: 0:21:22.127873
1023 time: 0:21:23.368686
1024 time: 0:21:24.602659
1025 time: 0:21:25.860556
1026 time: 0:21:27.107990
1027 time: 0:21:28.3393

1305 time: 0:27:19.560357
1306 time: 0:27:20.801704
1307 time: 0:27:22.078958
1308 time: 0:27:23.327217
1309 time: 0:27:24.555733
1310 time: 0:27:25.790302
1311 time: 0:27:27.025991
1312 time: 0:27:28.281046
1313 time: 0:27:29.539724
1314 time: 0:27:30.771047
1315 time: 0:27:32.038937
1316 time: 0:27:33.318206
1317 time: 0:27:34.558078
1318 time: 0:27:35.796250
1319 time: 0:27:37.022793
1320 time: 0:27:38.261408
1321 time: 0:27:39.530687
1322 time: 0:27:40.781343
1323 time: 0:27:42.015414
1324 time: 0:27:43.246496
1325 time: 0:27:44.508294
1326 time: 0:27:45.773426
1327 time: 0:27:47.019083
1328 time: 0:27:48.264382
1329 time: 0:27:49.498401
1330 time: 0:27:50.756406
1331 time: 0:27:52.016147
1332 time: 0:27:53.253787
1333 time: 0:27:54.487158
1334 time: 0:27:55.737713
1335 time: 0:27:56.994190
1336 time: 0:27:58.250498
1337 time: 0:27:59.504094
1338 time: 0:28:00.739343
1339 time: 0:28:02.002008
1340 time: 0:28:03.267907
1341 time: 0:28:04.510737
1342 time: 0:28:05.747626
1343 time: 0

...

37146 time: 14:54:07.098094
37147 time: 14:54:09.856717
37148 time: 14:54:12.591402
37149 time: 14:54:15.318110
37150 time: 14:54:18.084711
37151 time: 14:54:22.249572
37152 time: 14:54:24.985256
37153 time: 14:54:27.737895
37154 time: 14:54:30.483553
37155 time: 14:54:33.218238
37156 time: 14:54:35.974889
37157 time: 14:54:38.715536
37158 time: 14:54:41.529012
37159 time: 14:54:44.364429
37160 time: 14:54:47.097120
37161 time: 14:54:49.815880
37162 time: 14:54:52.558538
37163 time: 14:54:55.309158
37164 time: 14:54:58.042870
37165 time: 14:55:00.788505
37166 time: 14:55:03.534161
37167 time: 14:55:06.286800
37168 time: 14:55:09.039438
37169 time: 14:55:11.962620
37170 time: 14:55:14.693330
37171 time: 14:55:17.505796
37172 time: 14:55:20.284365
37173 time: 14:55:23.038023
37174 time: 14:55:25.765705
37175 time: 14:55:28.506376
37176 time: 14:55:31.235079
37177 time: 14:55:33.990708
37178 time: 14:55:36.727390
37179 time: 14:55:39.459096
37180 time: 14:55:42.209728
37181 time: 14:55:44

37439 time: 15:07:38.864111
37440 time: 15:07:41.590818
37441 time: 15:07:44.316529
37442 time: 15:07:47.061213
37443 time: 15:07:49.799864
37444 time: 15:07:52.521585
37445 time: 15:07:55.245301
37446 time: 15:07:57.973006
37447 time: 15:08:00.686771
37448 time: 15:08:03.406474
37449 time: 15:08:06.123209
37450 time: 15:08:08.864876
37451 time: 15:08:13.001812
37452 time: 15:08:15.726526
37453 time: 15:08:18.456742
37454 time: 15:08:21.175471
37455 time: 15:08:23.910162
37456 time: 15:08:26.621906
37457 time: 15:08:29.558053
37458 time: 15:08:32.294734
37459 time: 15:08:35.014460
37460 time: 15:08:37.859185
37461 time: 15:08:40.654968
37462 time: 15:08:43.416864
37463 time: 15:08:46.165975
37464 time: 15:08:48.928200
37465 time: 15:08:51.671994
37466 time: 15:08:54.406681
37467 time: 15:08:57.159320
37468 time: 15:08:59.903986
37469 time: 15:09:02.646643
37470 time: 15:09:05.381330
37471 time: 15:09:08.132971
37472 time: 15:09:10.868654
37473 time: 15:09:13.629294
37474 time: 15:09:16

37732 time: 15:21:11.405707
37733 time: 15:21:14.428614
37734 time: 15:21:17.168287
37735 time: 15:21:19.892003
37736 time: 15:21:22.652620
37737 time: 15:21:25.392293
37738 time: 15:21:28.128973
37739 time: 15:21:30.909537
37740 time: 15:21:33.649213
37741 time: 15:21:36.381902
37742 time: 15:21:39.126562
37743 time: 15:21:41.899146
37744 time: 15:21:44.655775
37745 time: 15:21:47.399976
37746 time: 15:21:50.134641
37747 time: 15:21:52.872319
37748 time: 15:21:55.610994
37749 time: 15:21:58.348673
37750 time: 15:22:01.097321
37751 time: 15:22:05.276146
37752 time: 15:22:08.021803
37753 time: 15:22:10.763470
37754 time: 15:22:13.581979
37755 time: 15:22:16.347568
37756 time: 15:22:19.109182
37757 time: 15:22:21.886755
37758 time: 15:22:24.615464
37759 time: 15:22:27.361114
37760 time: 15:22:30.096809
37761 time: 15:22:32.831483
37762 time: 15:22:35.568164
37763 time: 15:22:38.331774
37764 time: 15:22:41.060477
37765 time: 15:22:43.797157
37766 time: 15:22:46.531868
37767 time: 15:22:49

38025 time: 15:34:45.011324
38026 time: 15:34:47.742021
38027 time: 15:34:50.501641
38028 time: 15:34:53.223365
38029 time: 15:34:55.958049
38030 time: 15:34:58.674783
38031 time: 15:35:01.428419
38032 time: 15:35:04.166098
38033 time: 15:35:06.879840
38034 time: 15:35:09.614526
38035 time: 15:35:12.335249
38036 time: 15:35:15.157701
38037 time: 15:35:17.884410
38038 time: 15:35:20.615107
38039 time: 15:35:23.352784
38040 time: 15:35:26.087471
38041 time: 15:35:28.812184
38042 time: 15:35:31.552854
38043 time: 15:35:34.297513
38044 time: 15:35:37.012254
38045 time: 15:35:39.739959
38046 time: 15:35:42.523514
38047 time: 15:35:45.307070
38048 time: 15:35:48.105587
38049 time: 15:35:50.931029
38050 time: 15:35:53.685686
38051 time: 15:35:57.853516
38052 time: 15:36:00.583216
38053 time: 15:36:03.319897
38054 time: 15:36:06.074530
38055 time: 15:36:08.804229
38056 time: 15:36:11.524953
38057 time: 15:36:14.335437
38058 time: 15:36:17.060150
38059 time: 15:36:19.804817
38060 time: 15:36:22

KeyboardInterrupt: 

In [ ]:
generator.summary()

In [27]:
save_generator = keras.Sequential([generator])

In [28]:
save_generator.add(Lambda(lambda x: (x + 1) * 127.5))

## Convert to CoreML

In [29]:
save_generator.save('generator.h5')

In [68]:
! pip install coremltools

ERROR: Could not find a version that satisfies the requirement coremltools (from versions: none)
ERROR: No matching distribution found for coremltools


In [ ]:
! pip install keras==2.2.4
! pip install tensorflow==1.14.0

     |████████████████████████████████| 3.4MB 39kB/s eta 0:00:011
     |████████████████████████████████| 317kB 2.8MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


     |████████████████████████████████| 109.2MB 1.3MB/s 
     |████████████████████████████████| 491kB 42.1MB/s 
     |████████████████████████████████| 3.2MB 38.2MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [66]:
from coremltools.proto import NeuralNetwork_pb2

def convert_lambda(layer):
    params = NeuralNetwork_pb2.CustomLayerParams()

    # The name of the Swift or Obj-C class that implements this layer.
    params.className = "Lambda"

    # The desciption is shown in Xcode's mlmodel viewer.
    params.description = "Post process"

    return params

ModuleNotFoundError: No module named 'coremltools'

In [67]:
import coremltools
coreml_model = coremltools.converters.keras.convert('generator.h5', input_names='input', output_names='output', image_input_names='input', add_custom_layers=True, image_scale=1/127.5, gray_bias=-1, custom_conversion_functions={ "Lambda": convert_lambda })

# Saving the Core ML model to a file.
coreml_model.save('colorizer.mlmodel')

ModuleNotFoundError: No module named 'coremltools'

In [ ]:
import coremltools
import coremltools.proto.FeatureTypes_pb2 as ft 

spec = coremltools.utils.load_spec("colorizer.mlmodel")

In [ ]:
output = spec.description.output[0]
width = 512
import coremltools.proto.FeatureTypes_pb2 as ft
output.type.imageType.colorSpace = ft.ImageFeatureType.RGB
output.type.imageType.height = width
output.type.imageType.width = width

coremltools.utils.save_spec(spec, "Colorizer.mlmodel")

In [ ]:
#! pip uninstall keras
#! pip uninstall tensorflow
! pip install tensorflow
! pip install keras